##On-Device ML using Tensorflow Lite##
###Companion Colab Notebook for the Medium article by Roman Jaquez###
[asdfasd](dfdsfa)

In this tutorial we'll achieve the following:

Build and training a simple ML model that converts a value from celsius to

* Fahrenheit (from scratch) using a Google Colab notebook.
* Export our trained model out of Google Colab as a TensorFlow Lite model for on-device consumption

### Benefits of On-Device ML:###
* Low Latency: By having the ML model right on your device, you can feed it user-generated data (text, audio, video) in real-time.
* Keep data on-device: Great for user privacy - the data doesn't leave the device and you can perform inference locally.
* Works offline: Lack of reliance on a network connection or having to run a service in the cloud makes this attractive - all processing happens on the device.



###Building your ML Model in Google Colab###
Google Colab is free to use, and. you should use a valid Gmail account to access it.

The problem we will solve is to convert from Celsius to Fahrenheit, where the approximate formula is:

```𝑓=𝑐×1.8+32```

Of course, it would be simple enough to create a conventional Dart function that directly performs this calculation, but that wouldn't be machine learning.

Instead, we will give TensorFlow some sample Celsius values (0, 5, 10, 15, 20) and their corresponding Fahrenheit values (32, 41, 50, 59, 68). Then, we will train a model that figures out the above formula through the training process.

###Import Dependencies###
We need to import Tensorflow; use an alias - tf for short- as well as libraries such as numpy (commonly used in machine learning)

In [1]:
import tensorflow as tf
import numpy as np

###Define the training data###
We'll be performing Supervised Machine Learning - which is all about figuring out an algorithm given a set of inputs and outputs. We want a model that given the degree in Celsius, it yields the degree in Fahrenheit, so we will create two lists - one called celsius_features and fahrenheit_labels, type float - that we will use to train our model.

In [2]:
celsius_features = np.array([0, 5,  10, 15, 20, 25, 30, 35, 40,   45, 50,  55,  60, 65,  70, 75,   80,  85, 90, 95, 100],  dtype=float)
fahrenheit_labels = np.array([32, 41, 50, 59, 68, 77, 86, 95, 104, 113, 122, 131, 140, 149, 158, 167, 176, 185, 194, 203, 212],  dtype=float)

for i,c in enumerate(celsius_features):
  print("{} degrees Celsius = {} degrees Fahrenheit".format(c, fahrenheit_labels[i]))

0.0 degrees Celsius = 32.0 degrees Fahrenheit
5.0 degrees Celsius = 41.0 degrees Fahrenheit
10.0 degrees Celsius = 50.0 degrees Fahrenheit
15.0 degrees Celsius = 59.0 degrees Fahrenheit
20.0 degrees Celsius = 68.0 degrees Fahrenheit
25.0 degrees Celsius = 77.0 degrees Fahrenheit
30.0 degrees Celsius = 86.0 degrees Fahrenheit
35.0 degrees Celsius = 95.0 degrees Fahrenheit
40.0 degrees Celsius = 104.0 degrees Fahrenheit
45.0 degrees Celsius = 113.0 degrees Fahrenheit
50.0 degrees Celsius = 122.0 degrees Fahrenheit
55.0 degrees Celsius = 131.0 degrees Fahrenheit
60.0 degrees Celsius = 140.0 degrees Fahrenheit
65.0 degrees Celsius = 149.0 degrees Fahrenheit
70.0 degrees Celsius = 158.0 degrees Fahrenheit
75.0 degrees Celsius = 167.0 degrees Fahrenheit
80.0 degrees Celsius = 176.0 degrees Fahrenheit
85.0 degrees Celsius = 185.0 degrees Fahrenheit
90.0 degrees Celsius = 194.0 degrees Fahrenheit
95.0 degrees Celsius = 203.0 degrees Fahrenheit
100.0 degrees Celsius = 212.0 degrees Fahrenheit


###Some Machine Learning terminology###
* Feature - The input(s) to our model. In this case, a single value - the degrees in Celsius.
* Labels - The output our model predicts. In this case, a single value - the degrees in Fahrenheit.

###Create the model###
The next step is to create the model. A Dense network is the simplest TensorFlow model we can use since our problem is pretty is straightforward. This will be a network with a single layer, and a single neuron.
###Build a layer###
We'll call the layer layer_0 by using the tf.keras.layers.Dense and the following parameters:
* input_shape=[1]: this means the input to this layer is a single value - the shape of a one-dimensional array with one member. Since this is the first (and only) layer, the input shape is the input shape of the entire model. The single value is a floating point number, which represents the degrees Celsius.
* units=1: the number of neurons in the layer, or what defines how many internal variables the layer has to try to learn to solve the problem. Since this is the final layer, it is also the size of the model's output - a single floating point number representing degrees Fahrenheit.

In [3]:
layer_0 = tf.keras.layers.Dense(units=1, input_shape=[1])

Add the layer to a Sequential model, appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor:

In [4]:
model = tf.keras.Sequential([layer_0])

###Compile the model, with loss and optimizer functions###
You must compile the model before training occurs. During compilation of the model, you must provide the following to the model:
Loss function - A way of measuring how far off predictions are from the desired outcome. (The measured difference is called the "loss"). In our case, the "mean_squared_error".
Optimizer function - A way of adjusting internal values in order to reduce the loss; in our case, an optimizer that implements the Adam algorithm.

In [5]:
model.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(0.1))

###Train the model###
Train the model by calling fit on the model.

During training, the model takes in Celsius values, performs a calculation using the current internal variables (called "weights") and outputs values which are meant to be the Fahrenheit equivalent. Since the weights are initially set randomly, the output will not be close to the correct value.

The difference between the actual output and the desired output is calculated using the loss function, and the optimizer function directs how the weights should be adjusted.

This cycle of calculate, compare, adjust is controlled by the fit method. The first argument is the inputs, the second argument is the desired outputs. The epochs argument specifies how many times this cycle should be run, and the verbose argument controls how much output the method produces.

In [6]:
model.fit(celsius_features, fahrenheit_labels, epochs=2000, verbose=False)
print("Finished training the model")

Finished training the model


###Use the model to predict values###
Congrats! You've just trained a model that has been trained to learn the relationship between celsius_features and fahrenheit_labels. With that done, you can proceed and use the predict method in order to have it calculate the Fahrenheit degrees for a previously unknown Celsius degrees.

So, for example, if the Celsius value is 0, what do you think the Fahrenheit result will be? Take a guess before you run this code.

In [7]:
print(model.predict([0.0]))

1/1 [==============================] - 0s 169ms/step
[[31.341253]]


###Save and Export your Trained Model###
Let's get our model out of Google Colab for consumption.

But first let's do some configuration to accomplish this.

Save the model in the Tensorflow format (tf). The saved model contains:
* the model's configuration (architecture)
* the model's weights
* the model's optimizer state (if any)

Store the name of the path of our model in an environment variable called KERAS_MODEL_PATH, as such:

In [8]:
KERAS_MODEL_PATH = 'tf_keras_c2f.tf'

Save the model using the provided path in the tf format, as such:

In [9]:
model.save(KERAS_MODEL_PATH, save_format='tf')

With the saved model, now let's proceed to convert our Tensorflow model to a Tensorflow Lite format, for on-device consumption, using the TFLiteConverter utility; save it to your local Google Colab environment, from where you can later download it.

In [10]:
converter = tf.lite.TFLiteConverter.from_saved_model(KERAS_MODEL_PATH)
tflite_model = converter.convert()

# Save the TFLITE model
with open('modelc2f.tflite', 'wb') as f:
    f.write(tflite_model)

Download it by right-clicking on it and selecting Download.
Now you have a fully-trained, simple, yet fully-functional ML model that you can use anywhere you can consume a Tensorflow Lite model; all it takes is finding the proper packages and SDK to read the .tflite file, feed it the expected input and just wait for magic to happen!

Check out how I developed a Flutter app that consumes this trained ML TFLite model by following this link.

### Please don't forget to follow me on social media:

- On Twitter ([@drcoderz](https://www.twitter.com/drcoderz))
- On YouTube ([Roman Just Codes](https://www.youtube.com/channel/UCKsp3r1ERjCpKJtD2n5WtPg))
- On my [Personal Portfolio](https://romanjustcodes.web.app/)
- On [Medium](https://medium.com/@romanejaquez)
- On [LinkedIn](https://www.linkedin.com/in/roman-jaquez-8941a424/)

